# Exercise 6.1 Mapping EFK

## Theoretical background

In the previous lessons we have applied two algorithms: *Least Squares* and the *Extended Kalman Filter* to the Localization problem. Both algorithms can also be applied to the Mapping problem, with some minimal changes.

In contrast to previous lessons, we do not have a perfect map of the environment, but we do have perfect knowledge of the robots pose. The absence of a perfect map of the environment is a common occurence in the field of mobile robots, some times no map is provided, or even if we had a map, it may contain wrong information.
To simplify the problem we will assume the environment is unchanging, which is false in most of the useful applications.

This lesson will be centered around the EKF algorithm and how it is applied to mapping. We will assume you already know how to implement this algorithm, focusing only in the main differences of implementation:

**First**, we ought to consider the consider the *state*, which in this case is a **vector of landmarks**. This takes the form of:

$$m = \textit{xEst} = [x_0, y_0, \underbrace{x_1, y_1}_{\text{Position of}\\\text{landmark 1}}, \dots]^T$$

As we can see, we do not consider the orientation of the landmarks, therefore if $n$ is the number of seen landmarks, $len(m) = 2n$.

**Second**, we need the covariace matrix of the state, which we can deduce from how the observations are taken:

- A range-bearing sensor generates a measurement with a covariance: $\Sigma_{r\theta} = \begin{bmatrix} \sigma^2_r & 0 \\ 0 & \sigma^2_\theta  \end{bmatrix}$
- The contents of a map need to be transformed to the world frame: $\Sigma_{xy} = J \Sigma_{r\theta} J^T$. Just as we learned in Lesson 4. (Sensing).
- Finally we need to combine the covariances to build the map's one.:

  $$
  \begin{bmatrix}
    [\Sigma^0_{xy}]_{2 \times 2} & \cdots & 0_{2 \times 2} \\
    \vdots & \ddots & \vdots \\
    0_{2 \times 2} & \cdots & [\Sigma^n_{xy}]_{2 \times 2}
  \end{bmatrix}_{2n \times 2n}
  $$
  
**Third**, we face an additional problem we did not have on the Location problem, this is we do not observe the whole set of landmarks each time, furthemore we do not know the amount of landmarks there exists. We need to add the landmarks once we see them for the first time.

For your convenience, it is included here the slide illustrating how the algorithm performs once the sensor takes a measurement to a landmark.

![](images/fig6-1-1.png)

In this case we make the assumption that the **robot’s pose is known** (without uncertainty)
and we want to estimate the pdf of the location of a number of landmarks that are
present in the robot’s surroundings, utilizing for that a **noisy sensor**.

In [ ]:
%matplotlib notebook

import sys
import time

import numpy as np
from numpy import random
from scipy import linalg
import matplotlib
import matplotlib.pyplot as plt

from utils.tcomp import tcomp
from utils.AngleWrap import AngleWrap
from utils.DrawRobot import DrawRobot
from utils.PlotEllipse import PlotEllipse
from utils.Jacobians import J2
from utils.unit6.MapCanvas import MapCanvas

## 1. Complete the code.

The algorithm has gaps at some key places, so your first goal is to fill them with the appropriate code. For that, first review the code that is written and understand what is going on. Concretely, your mission is to implement the Jacobians computation, as well as some stuff related to the measurements.

- Take a look at the class that will simulate our robot. It is similar to robots in previous practices as it can move and observe some landmarks. There is no uncertainty in its movement.

In [ ]:
class EFKMappingRobot():
    def __init__(self, true_pose, sigma_r, sigma_theta, n_features):
        # Robot description
        self.true_pose = true_pose
        self.Q = np.diag([sigma_r, sigma_theta])**2
        
        # Map -- Initially empty
        self.xEst = np.empty((0, 0))
        self.PEst = np.empty((0, 0))
        self.QEst = 1.0*self.Q
        
        self.MappedFeatures = -1*np.ones((n_features,1), int)
        
    def step(self, u):
        self.true_pose = tcomp(self.true_pose, u)
        
    def observe(self, idx, world, noisy=True):
        """ Generate a observation of a feature in our world
        
            Args:
                world: Complete map of all landmarks in the world
                idx: Landmark to observe (index in world matrix)
                noisy: Add noise to z, prorportional to self.Q
                
            Returns:
                z: One range and bearing observation
        """
        
        z = np.empty((2, 1))
        delta = world[:, [idx]] - self.true_pose[0:2, :]
        
        # Range
        z[0, :] = np.sqrt(np.sum(delta**2))
        # Bearing
        z[1, :] = np.arctan2(delta[1, 0],delta[0, 0]) - self.true_pose[2, 0]
        z[1, :] = AngleWrap(z[1, :])
        
        if noisy:
            z = z + np.sqrt(self.Q)@random.rand(2,1)
            
        return z
    
    def get_random_observation(self, world, noisy=True):
        iFeatures = world.shape[1]
        iFeature = random.randint(iFeatures)
        z = self.observe(iFeature, world, noisy)
        return z, [iFeature]
        

- In order to build the EFK algorithm we are going to need to calculate some jacobians. Complete the following functions using the formulas from the theory lessons.

In [ ]:
def GetObsJacs(xPred, xFeature):
    """ Calculate the jacobian of the observation model.
        
        Needed to update a landmark we have already seen.
        Hint. Similar to the one described in unit 5 (Localization)
        
        Args:
            xPred: True pose of our robot.
            xFeature: Estimated pose of a landmark in our map. World p.o.v in cartesian coordinates.
                Does not contain an angle.
        
        Return:
            2x2 matrix containing the corresponding jacobian.
    """
    #TODO
    raise NotImplementedError
    
    return jHxf

def GetNewFeatureJacs(Xv, z):
    """ Calculate the jacobian for transforming an observation to the world frame
    
        Args:
            Xv: True pose of our robot
            z: Observation of a landmark. In polar coordinates from the p.o.v. of our robot.
        
        Returns:
            2x2 matrix containing the corresponding jacobian.
    """
    #TODO
    raise NotImplementedError
    
    return jGz


- The EFK algorithm is divided in 2 steps: prediction and update. Depending on whether our robot has seen this landmark before or not the algorithm beheaves differently.

In [ ]:
def EFKLocalization(robot: EFKMappingRobot, z, iFeature):
    """ EFK algorithm for mapping
        
        Args:
            robot: Robot base (contains state map: xEst, PEst)
            z: Observation of a landmark
            iFeature: Index of z in the world map
    
        Returns: Nothing. But it modifies the state in robot
    """
    
    # We assume that the map is static (the state transition model of
    xPred = robot.xEst
    PPred = robot.PEst

    # Check if feature observed is in map
    if robot.MappedFeatures[iFeature] > -1:
        # Find out where it is in state vector
        FeatureIndex = robot.MappedFeatures[iFeature[0], 0]
        
        # xFeature is the current estimation of the position of the
        # landmard "FeatureIndex"
        xFeature = xPred[FeatureIndex:FeatureIndex+2]
        
        # TODO Predicts the observation
        zPred = None # Hint: use robot.observe function

        # Get observation Jacobians
        jHxf = GetObsJacs(robot.true_pose, xFeature)
        
        # Fill in state jacobian
        # (the jacobian is zero except for the observed landmark)
        jH = None
        
        #
        # Kalman update
        #

        Innov = z-zPred # Innovation
        Innov[1] = AngleWrap(Innov[1])
        S = jH@PPred@jH.T + robot.QEst
        K = PPred@jH.T@linalg.inv(S) # Gain
        robot.xEst = xPred + K@Innov
        robot.PEst = PPred - K@S@K.T

        #ensure P remains symmetric
        robot.PEst = 0.5*(robot.PEst+robot.PEst.T)
        
    else:
        # This is a new feature, so add it to the map
        nStates = xPred.size 
        
        # The observation is in the local frame of the robot, it has to
        # be translated to the global frame
        xFeature = np.vstack([
            z[0, 0]*np.cos(z[1, 0] + robot.true_pose[2, 0]),
            z[0, 0]*np.sin(z[1, 0] + robot.true_pose[2, 0])
        ])

        #TODO
        xFeature = None

        # Add it to the current state
        if nStates == 0:
            robot.xEst = xFeature
        else:
            robot.xEst = np.vstack([robot.xEst, xFeature]) #Each new feature two new rows

        # Compute the jacobian
        jGz = GetNewFeatureJacs(robot.true_pose, z) #Dimension 2x2
        if nStates != 0:
            # note we don't use jacobian w.r.t vehicle since the pose doesn’t have uncertainty
            M = np.vstack([  
                    np.hstack([np.eye(nStates), np.zeros((nStates, 2))]),
                    np.hstack([np.zeros((2, nStates)), jGz])
                ])
        else:
            M = jGz
            
        robot.PEst = M@linalg.block_diag(robot.PEst, robot.QEst)@M.T

        #This can also be done directly PEst = [PEst,zeros(nStates,2);
                                                # zeros(2,nStates),
                                                # jGz*QEst*jGz']

        #remember this feature as being mapped: we store its ID for the state vector
        robot.MappedFeatures[iFeature] = robot.xEst.size-2
        #Always an odd number

- Take a look to the following demo and use it to test your implementation with the parameters given below. You are not meant to modify the `main()` at any point of the exercise.

In [ ]:
def main(robot,
         Map,
         nFeatures,
         mode='non_stop',
         logger=None,
         nSteps=100, # Number of motions
         turning= 40, # Number of motions before turning (square path)
         print_each=2):
    
    %matplotlib notebook
    if mode == 'step_by_step':
        matplotlib.use('TkAgg')

    # storing the number of times a features has been seen
    # also store the handler to the graphical info shown
    canvas = MapCanvas(nFeatures)
    
    canvas.ax.plot(Map[0, :], Map[1, :], 'g*')
    hObsLine = canvas.ax.plot([0,0], [0,0], linestyle=':')
    
    # Control action
    u = np.zeros((3, 1))
    u[0] = (2.*MapSize/1.5)/turning
    u[1] = 0.
    
    # Start the loop!
    for k in range(nSteps):
        #
        # Move the robot
        #
        u[2]=0.
        if k%turning == turning-1:
            u[2] = np.pi/2
        
        robot.step(u) # Perfectly known robot pose
        
        z, iFeature = robot.get_random_observation(world=Map)
        
        # Update the "observedtimes" for the feature and plot the reading
        canvas.increment_observed_times(iFeature)
        canvas.PlotNumberOfReadings(robot.true_pose, iFeature, Map)
        
        EFKLocalization(robot, z, iFeature)
        
        # Log important values
        if logger is not None:
            logger.log(k, robot, Map)
        
        # Drawings
        if k%print_each == print_each-1:
            DrawRobot(canvas.fig, canvas.ax,robot.true_pose, 'r')#plot(xVehicleTrue(1),xVehicleTrue(2),'r*')
            canvas.DoMapGraphics(robot) # Draw estimated poitns (in black) and ellipses
            plt.axis([-MapSize-5, MapSize+5, -MapSize-5, MapSize+5]) # Set limits again
            plt.draw()
            
            if mode == 'step_by_step':
                plt.waitforbuttonpress(-1)
            elif mode == 'visualize_process':
                plt.pause(0.2)
            elif mode == 'non_stop':
                pass # non stop!

    # Final drawings
    %matplotlib inline
    if logger is not None:
        logger.plot()

## 2. Consider that only 1 landmark exists.

Set the variable nFeatures to 1. Execute the program and modify the `main()` to show the content of the vector of states `xEst` and the covariance matriz `Pest` each 5 iterations. What dimensions do they have?

<figure style="text-align:center">
  <img src="images/fig6-1-2.png" width="400" alt="">
  <figcaption>
      Fig. 1: Execution of the EKF algorithmn for mapping (only one landmark). <br/>
      it shows the true pose (in red), <br/>
      the real pose of the landmark (as a green star), <br/>
      and the estimation from the EKF algorithm (pose and confidence ellipse).
  </figcaption>
</figure>

In [ ]:
#mode = 'step_by_step'
mode = 'visualize_process'
#mode = 'non_stop'

# WORLD MAP
# Num features/landmarks considered within the map
nFeatures = 1
# Generation of the map
MapSize = 100
Map = MapSize*random.rand(2,nFeatures)-MapSize/2

# ROBOT
# Covariances for our very bad&expensive sensor (in the system <d,theta>)
Sigma_r = 8.0
Sigma_theta = 7*np.pi/180
# Initial robot pose
xVehicleTrue = np.vstack([-MapSize/1.5, -MapSize/1.5, 0.]) # We know the exact robot pose at any moment

robot = EFKMappingRobot(xVehicleTrue, Sigma_r, Sigma_theta, nFeatures)

main(robot, Map ,nFeatures, mode=mode)

## 3. - Repeat the last point employing 5 landmarks.

Explain why and how the content of the variables `xEst` and `Pest` has change. Show also, each 5 iterations, the content of the jacobian of the observation (`jH`). What structure does the matrix of covariances have? Is there any kind of correlation among the observations of different landmarks?

<figure style="text-align:center">
  <img src="images/fig6-1-3.png" width="400" alt="">
  <figcaption>
      Fig. 2: Execution of the EKF algorithmn for mapping (multiple landmarks). <br/>
      Same as in Fig 1., each landmark is accompanied by a number of times observed.
  </figcaption>
</figure>

In [ ]:
#mode = 'step_by_step'
mode = 'visualize_process'
#mode = 'non_stop'

# WORLD MAP
# Num features/landmarks considered within the map
nFeatures = 5
# Generation of the map
MapSize = 100
Map = MapSize*random.rand(2,nFeatures)-MapSize/2

# ROBOT
# Covariances for our very bad&expensive sensor (in the system <d,theta>)
Sigma_r = 8.0
Sigma_theta = 7*np.pi/180
# Initial robot pose
xVehicleTrue = np.vstack([-MapSize/1.5, -MapSize/1.5, 0.]) # We know the exact robot pose at any moment

robot = EFKMappingRobot(xVehicleTrue, Sigma_r, Sigma_theta, nFeatures)

main(robot, Map ,nFeatures, mode=mode)

# 4. Results of the mapping process. 

We want to visualize how well the EFK algoritm performs. In the demo, there is an input parameter called logger, which is meant to store some information each loop and plot it at the end.

Implement the following class to store the determinant of the covariance matrix of each landmark and the error in the fitting along the algorithm iterations. Plot it for the case of 5 landmarks.

In [ ]:
import pandas as pd

class Logger():
    """ Logs info about the covariance and error of a map.
    
        Attrs:
            n_features: Number of features in the world.
            log_error: Matrix to store the error in the fitting for each landmark.
            log_det: Matrix to store the determinant of the covariance matrix for each landmark.
    
    """
    def __init__(self, n_steps, n_features):
        """ Initializes each matrix to log the information
        
            Args:
                n_steps: Maximum number of steps our robot will take.
                n_features: Number of features in the world.
        """
        self.n_features = n_features
        self.log_error = np.empty((n_steps,n_features))
        self.log_det = np.empty((n_steps,n_features))
            
    def log(self, k: int, robot: EFKMappingRobot, Map: np.ndarray):
        """ Computes relevant info about the error and covariances.
        
            It is called once per loop in the demo.
        
            Args:
                k: Number of iteration we are at. Range: [0, n_steps)
                robot: 
                Map:
        """
        #TODO
        raise NotImplementedError
        
    def plot(self):
        """ Plot all relevant figures. It is called at the end of the demo"""
        fig1 , ax1 =plt.subplots(1, 1, sharex=True)
        fig2 , ax2 =plt.subplots(1, 1, sharex=True)
        fig2.tight_layout()
        fig1.tight_layout()

        df1 = pd.DataFrame(data= self.log_error, columns = ['Landmark {}'.format(i) for i in range(self.n_features)])
        ax1.set_title('Error between map and est')
        df1.plot(ax = ax1)
        df2 = pd.DataFrame(data=np.log(self.log_det), columns=['Landmark {}'.format(i) for i in range(self.n_features)])
        ax2.set_title('Det. of covar.')
        df2.plot(ax = ax2)

In [ ]:
#mode = 'step_by_step'
mode = 'visualize_process'
#mode = 'non_stop'

# WORLD MAP
# Num features/landmarks considered within the map
nFeatures = 5
# Generation of the map
MapSize = 100
Map = MapSize*random.rand(2,nFeatures)-MapSize/2

# ROBOT
# Covariances for our very bad&expensive sensor (in the system <d,theta>)
Sigma_r = 8.0
Sigma_theta = 7*np.pi/180
# Initial robot pose
xVehicleTrue = np.vstack([-MapSize/1.5, -MapSize/1.5, 0.]) # We know the exact robot pose at any moment

robot = EFKMappingRobot(xVehicleTrue, Sigma_r, Sigma_theta, nFeatures)

nSteps=100
logger = Logger(n_features=nFeatures, n_steps=nSteps)

main(robot,
     Map,
     nFeatures,
     logger=logger,
     mode='non_stop',
     nSteps=nSteps)